In [1]:
paths.dir("./")

{
  1 : ..
  2 : Code
  3 : .
  4 : DQN_Batch_CNN.ipynb
  5 : data
  6 : .gitignore
  7 : .git
  8 : Presentation
  9 : README.md
  10 : .ipynb_checkpoints
  11 : Paper
}


In [2]:
require 'os'
require 'nn'
require 'rnn'
require 'optim'
-- require 'cunn'
-- require 'cunnx'
-- require 'cutorch'
require 'parallel'

dl = require 'dataload'

In [3]:
dofile("Code/utils.lua")
dofile("Code/utilsNNbatch.lua")
dofile("Code/Utils/load_cnn.lua")

...Utils file loaded	


...Utils file loaded	


In [4]:
outputpath = '/home/francisco/GitHub/DQN-Event-Summarization/data/training/'

In [5]:
torch.setnumthreads(torch.getnumthreads())

In [6]:
queries, sentences, trueSummaries = loadCNN(outputpath, 500)

In [22]:
vocab_size = 20001
learning_rate=1e-5
embDim = 50
gamma=0
batch_size = 25
mem_multiplier = 1
cuts = 4
endexplorerate=0.8
base_explore_rate=0.1
nepochs = 100
epsilon = 1
print_perf=false
adapt=false
adapt_lambda = 0.25
usecuda= false
seedval= 420

In [20]:
if usecuda then
    Tensor = torch.CudaTensor
    LongTensor = torch.CudaLongTensor   
    ByteTensor = torch.CudaByteTensor
    maskLayer = nn.MaskedSelect():cuda()
    print("...running on GPU")
else
    Tensor = torch.Tensor
    LongTensor = torch.LongTensor
    ByteTensor = torch.ByteTensor
    maskLayer = nn.MaskedSelect()
    print("...running on CPU")
end

...running on CPU	


In [16]:
SKIP = 1
SELECT = 2

k = #trueSummaries
n = queries[1]:size(1)
q = queries[1]:size(2)
n_s = #queries

optimParams = { learningRate = learning_rate }
delta = cuts / nepochs
end_baserate = torch.round(nepochs * endexplorerate )


qTokens = {}
for i=1, n do
    qTokens[i] = Tokenize({trueSummaries[1][i]:totable()}, false)[1]
end

-- Building the model
model = buildModel('bow', vocab_size, embDim, 'f1', adapt, usecuda)
params, gradParams = model:getParameters()

if adapt then 
    criterion = nn.ParallelCriterion():add(nn.MSECriterion()):add(nn.BCECriterion())
    criterion["weights"] = {1, adapt_lambda}
else 
    criterion = nn.MSECriterion()
end 


Running bag-of-words model to learn f1	


In [17]:
vocabsize = 0
for i=1, 124 do
    vocabsize = math.max(queries[i]:max(), sentences[i]:max(), trueSummaries[i]:max(), vocabsize)
end

In [23]:
print(string.format("Running model with %i queries and %i sentences", n, n_s))
qValues = {}
qActions = {}
qPreds = {}
rewards = {}
lossfull = {}
rouguef1 = {}
rougue_scores = {}

totalPredsummary = LongTensor(n, n_s * k):fill(0)

memfull = false
curr_memsize = 0
memsize = n * n_s * mem_multiplier
queryMemory = Tensor(memsize, q):fill(0)
qActionMemory = Tensor(memsize, 2):fill(0)
predSummaryMemory = Tensor(memsize, n_s * k):fill(0)
sentenceMemory = Tensor(memsize, k):fill(0)
sentencetp1Memory  = Tensor(memsize, k):fill(0)
predSummarytp1Memory = Tensor(memsize, n_s * k):fill(0)
qPredsMemory = Tensor(memsize, 2):fill(0)
qValuesMemory = Tensor(memsize, 1):fill(0)
rewardMemory = Tensor(memsize, 1):fill(0)
totalPreds = Tensor(n, 1):fill(0)


Running model with 500 queries and 125 sentences	


In [24]:
for i = 1, n_s do
    qPreds[i] = Tensor(n, 2):fill(0) 
    qValues[i] = Tensor(n, 1):fill(0)
    qActions[i] = Tensor(n, 2):fill(0)
    rewards[i] = Tensor(n, 1):fill(0)
    rougue_scores[i] = Tensor(n, 1):fill(0)
    if adapt then
        regPreds[i] = Tensor(n, 1):fill(0)
    end        
end 

In [25]:
for i=1, n_s do
    qPreds[i]:fill(0)
    qValues[i]:fill(0)
    qActions[i]:fill(0)
    rewards[i]:fill(0)
    rougue_scores[i]:fill(0)
    totalPredsummary:fill(0)
    if adapt then
        regMemory[i]:fill(0)
    end        
end

In [36]:
totalPreds:fill(0)
start_row = 1
end_row = batch_size
c = 1

In [38]:
i = 1

In [39]:
for start_row=1, n, batch_size do                 
    end_row = c * batch_size

    if end_row > n then 
        end_row = n
    end
    totalPreds[{{start_row, end_row}}]:copy(
        model:forward({
            queries[i][{{start_row, end_row}}], 
            sentences[i][{{start_row, end_row}}], 
            totalPredsummary[{{start_row, end_row}}]
        })
    )
    c = c + 1
end

[string "for start_row=1, n, batch_size do            ..."]:7: inconsistent tensor size at /tmp/luarocks_torch-scm-1-9784/torch7/lib/TH/generic/THTensorCopy.c:17
stack traceback:
	[C]: in function 'copy'
	[string "for start_row=1, n, batch_size do            ..."]:7: in main chunk
	[C]: in function 'xpcall'
	/home/francisco/torch/install/share/lua/5.1/itorch/main.lua:210: in function </home/francisco/torch/install/share/lua/5.1/itorch/main.lua:174>
	/home/francisco/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...francisco/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...francisco/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...francisco/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/francisco/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x00405d50: 

In [42]:
queries[i][{{start_row, end_row}}]:size(), sentences[i][{{start_row, end_row}}]:size(),  totalPredsummary[{{start_row, end_row}}]:size()

 25
 19
[torch.LongStorage of size 2]

  25
 662
[torch.LongStorage of size 2]

    25
 15625
[torch.LongStorage of size 2]

